In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from pathlib import Path
import statsmodels.api as sm
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# === CONFIGURACIÓN ===
DB = "../database/dodo_supermercado.db"
OUT = Path("../outputs")


def nombre_archivo_limpio(txt: str) -> str:
    return (
        str(txt)
        .lower()
        .replace(" ", "_")
        .replace("·", "_")
        .replace("ó", "o")
        .replace("á", "a")
        .replace("é", "e")
        .replace("í", "i")
        .replace("ú", "u")
    )

In [ ]:
# === CARGA DE DATOS ===
def carga_series(ciudad: str, categoria: str):
    sql = """
    WITH base AS (
        SELECT v.Fecha, v.Cantidad_Vendida, v.Id_Tienda, v.Categoria,
               i.Precio, i.Promocion, i.Descuento_pct, s.Ciudad
        FROM Ventas v
        JOIN Inventario i ON v.Id_Tienda = i.Id_Tienda AND v.Fecha = i.Fecha AND v.Categoria = i.Categoria
        JOIN Supermercado s ON v.Id_Tienda = s.Id_Tienda
        WHERE v.Fecha >= "2023-01-01" AND v.Fecha < "2025-01-01"
    )
    SELECT * FROM base
    WHERE LOWER(Ciudad) = ? AND LOWER(Categoria) = ?
    ORDER BY Fecha;
    """
    with sqlite3.connect(DB) as conn:
        df = pd.read_sql_query(
            sql, conn, params=(ciudad.lower(), categoria.lower()), parse_dates=["Fecha"]
        )
    df = df.set_index("Fecha").asfreq("D")
    y = df["Cantidad_Vendida"].astype(int)
    X = df[["Precio", "Promocion", "Descuento_pct"]].copy()
    X["Precio"] = X["Precio"].ffill().bfill().fillna(0)
    X["Promocion"] = X["Promocion"].fillna(0)
    X["Descuento_pct"] = X["Descuento_pct"].fillna(0)
    y, X = y.align(X, join="inner", axis=0)
    return y, X

In [ ]:
# === ENTRENAMIENTO DEL MODELO ===
def entrenar_sarimax(y, X):
    model = SARIMAX(
        y,
        exog=X,
        order=(1, 1, 1),
        seasonal_order=(1, 0, 1, 7),
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    return model.fit(disp=False)


def forecast_year(results, X_hist, year=2025):
    rng = np.random.default_rng(42)
    fut_idx = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="D")
    steps = len(fut_idx)
    tail = X_hist.tail(30)
    p_med = float(tail["Precio"].median())
    promo_p = float(tail["Promocion"].mean())
    desc_m = float(tail["Descuento_pct"].mean())

    drift_lineal = 1.0 + 0.02 * np.linspace(0, 1, steps)
    ruido_precio = rng.normal(0.0, 0.05, steps)
    precio_fut = p_med * drift_lineal * (1.0 + ruido_precio)
    precio_fut = np.maximum(precio_fut, 0.0)

    dow = pd.Series(fut_idx).dt.dayofweek.to_numpy()
    boost = np.where(dow >= 5, 0.10, 0.0)
    prob_promo = np.clip(promo_p + boost, 0.0, 1.0)
    promo_fut = rng.binomial(n=1, p=prob_promo, size=steps).astype(float)

    ruido_desc = rng.normal(0.0, 0.025, steps)
    desc_fut = np.clip(desc_m * (1.0 + ruido_desc), 0.0, 0.5)

    X_fut = pd.DataFrame(
        {"Precio": precio_fut, "Promocion": promo_fut, "Descuento_pct": desc_fut},
        index=fut_idx,
    )

    fc_obj = results.get_forecast(steps=steps, exog=X_fut)
    return fc_obj.predicted_mean, fc_obj.conf_int(alpha=0.05)

In [ ]:
# === VISUALIZACIÓN ===
def graficar_prediccion(
    historico, pred_2025, ci_2025, titulo, promedio_2024=None, fname=None
):
    x = pd.concat([historico, pred_2025]).index
    plt.figure(figsize=(18, 12))
    plt.plot(historico, label="Histórico 2023–2024", color="black")
    plt.plot(pred_2025, "--", label="Proyección 2025", color="blue")
    plt.fill_between(
        pred_2025.index,
        ci_2025.iloc[:, 0],
        ci_2025.iloc[:, 1],
        alpha=0.2,
        label="IC 95%",
        color="blue",
    )
    plt.axvline(
        pd.Timestamp("2025-01-01"),
        color="red",
        linestyle="--",
        label="Inicio Proyección",
    )
    if promedio_2024 is not None:
        plt.axhline(
            promedio_2024,
            color="red",
            linestyle=":",
            label=f"Promedio Diario 2024: {promedio_2024:.1f}",
        )
    plt.title(titulo)
    plt.xlabel("Fecha")
    plt.ylabel("Cantidad Vendida")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    if fname:
        plt.savefig(fname, dpi=150, bbox_inches="tight")
    plt.show()

In [ ]:
# === CASOS DE ESTUDIO ===
casos = [("arica", "bebidas"), ("antofagasta", "congelados")]

for ciudad, categoria in casos:
    y, X = carga_series(ciudad, categoria)
    print(f"\n=== {ciudad.upper()} · {categoria.upper()} ===")
    modelo = entrenar_sarimax(y, X)

    # Métricas 2024 para comparación visual
    y_2024 = y[(y.index >= "2024-01-01") & (y.index <= "2024-12-31")]
    promedio_2024 = y_2024.mean()
    std_2024 = y_2024.std()
    cv_2024 = (std_2024 / promedio_2024) * 100

    print(f"Promedio diario 2024: {promedio_2024:.0f}")
    print(f"Desviación estándar 2024: {std_2024:.2f}")
    print(f"Coef. de variación 2024: {cv_2024:.2f}%")

    pred_mean_2025, ci_2025 = forecast_year(modelo, X, year=2025)

    OUT.mkdir(parents=True, exist_ok=True)
    base = f"{nombre_archivo_limpio(ciudad)}_{nombre_archivo_limpio(categoria)}"
    output_file = OUT / f"{base}_prediccion_2025.png"

    graficar_prediccion(
        y,
        pred_mean_2025,
        ci_2025,
        f"SARIMAX · {ciudad.title()} - {categoria.title()} (2023–2024 histórico vs 2025 proyección)",
        promedio_2024=promedio_2024,
        fname=output_file,
    )

    print(f"Gráfico guardado en: {output_file}")

In [ ]:
# ================================================================
# EVALUACIÓN DEL MODELO SARIMAX (VALIDACIÓN CON DATOS REALES 2024) CON EXOGENAS
# ================================================================
def evaluar_sarimax(
    y,
    X,
    fecha_split="2024-01-01",
    order=(1, 1, 1),
    seasonal_order=(1, 0, 1, 7),
):
    """
    Evalúa el modelo SARIMAX usando 2024 como conjunto de test.
    - Entrena con datos anteriores a fecha_split.
    - Predice el tramo del 2024 usando exógenas reales.
    - Devuelve RMSE, MAE, MAPE y las predicciones.
    """

    # 1) Separar train/test por fecha
    mask_train = y.index < fecha_split
    y_train, y_test = y[mask_train], y[~mask_train]
    X_train, X_test = X[mask_train], X[~mask_train]

    # 2) Entrenamiento modelo SARIMAX
    model = SARIMAX(
        y_train,
        exog=X_train,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    results = model.fit(disp=False)

    # 3) Predicción sobre el tramo real del 2024
    pred_obj = results.get_prediction(
        start=y_test.index[0],
        end=y_test.index[-1],
        exog=X_test,
    )

    pred_mean = pred_obj.predicted_mean
    pred_ci = pred_obj.conf_int(alpha=0.05)

    # 4) Calcular métricas
    y_true = y_test.reindex(pred_mean.index)
    err = y_true - pred_mean

    rmse = float(np.sqrt((err**2).mean()))
    mae = float(np.abs(err).mean())
    mape = float((np.abs(err / y_true.replace(0, np.nan))).mean() * 100)

    return {
        "results": results,
        "y_test": y_true,
        "pred": pred_mean,
        "ci": pred_ci,
        "rmse": rmse,
        "mae": mae,
        "mape": mape,
    }


# ================================================================
# EJECUCIÓN: Evaluar el modelo para cada caso (2024 real)
# ================================================================
casos = [
    ("arica", "bebidas"),
    ("antofagasta", "congelados"),
]

for ciudad, categoria in casos:
    print(f"   Evaluación del Modelo - {ciudad.title()} · {categoria.title()}")
    print("===============================================")

    # Cargar series ya procesadas por tu función original
    y, X = carga_series(ciudad, categoria)

    # Ejecutar evaluación con 2024 como test
    eval_res = evaluar_sarimax(y, X, fecha_split="2024-01-01")

    print(f"RMSE (2024): {eval_res['rmse']:.2f}")
    print(f"MAE  (2024): {eval_res['mae']:.2f}")
    print(f"MAPE (2024): {eval_res['mape']:.2f}%")

    # -----------------------------------------------------------
    # Gráfico: Real 2024 vs Predicción 2024 (Backtesting)
    # -----------------------------------------------------------
    plt.figure(figsize=(18, 6))
    plt.plot(eval_res["y_test"], label="Real 2024", color="black")
    plt.plot(eval_res["pred"], label="Predicho SARIMAX", color="blue", linestyle="--")
    plt.fill_between(
        eval_res["pred"].index,
        eval_res["ci"].iloc[:, 0],
        eval_res["ci"].iloc[:, 1],
        alpha=0.2,
        color="blue",
        label="IC 95%",
    )
    plt.title(f"SARIMAX - Evaluación 2024 · {ciudad.title()} - {categoria.title()}")
    plt.xlabel("Fecha")
    plt.ylabel("Cantidad Vendida")
    plt.grid(True)
    plt.legend()
    plt.show()